In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *
import pandas as pd

In [3]:
# Download the data - 106 GB !!!
# !kaggle competitions download -c siim-isic-melanoma-classification

In [4]:
path = Config.data_path()/'melclass'
path.mkdir(parents=True, exist_ok=True)
path
bs = 64

In [5]:
data = ImageDataBunch.from_csv(
    path, 
    'jpeg/train', 
    csv_labels='subset.csv', # use full training data
    suffix='.jpg', 
    valid_pct=0.5,
    fn_col=0,
    label_col=7,
    size=224, # 224
    bs=64,
    ds_tfms=get_transforms()
).normalize(imagenet_stats) # need to use imagenet stats because this is the one the model was trained with

In [7]:
learn = cnn_learner(data, models.resnet50, metrics=[error_rate, AUROC()])
learn.load('melclass-stage-2-resnet50-full');

Learner(data=ImageDataBunch;

Train: LabelList (876 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
0,0,0,0,0
Path: /home/jupyter/.fastai/data/melclass;

Valid: LabelList (876 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
0,0,0,0,1
Path: /home/jupyter/.fastai/data/melclass;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [8]:
learn.validate(metrics=[error_rate, AUROC()])

[0.6310024, tensor(0.1587), tensor(0.9555)]

In [16]:
preds, y = learn.get_preds(ds_type = data)

In [17]:
preds

tensor([[9.9970e-01, 3.0326e-04],
        [9.9870e-01, 1.2997e-03],
        [9.9962e-01, 3.8370e-04],
        ...,
        [9.9970e-01, 2.9848e-04],
        [9.6960e-01, 3.0397e-02],
        [3.8906e-02, 9.6109e-01]])

In [18]:
imagenames = [str(l).split('/')[-1].replace('.jpg','') for l in data.items]
predcol = [p for p in preds.numpy()[:,1]]
df_preds = pd.DataFrame({'image_name':imagenames, 'prob':predcol, 'modclass': y.numpy()})
df_preds.head()

,image_name,prob,modclass
0,ISIC_0355023,0.000303,0
1,ISIC_7508527,0.001300,0
2,ISIC_2844085,0.000384,0
3,ISIC_2541969,0.000424,0
4,ISIC_2324728,0.000289,0


In [19]:
df = pd.read_csv(path/'subset.csv')
dfout = df.merge(df_preds, on='image_name')
dfout.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,prob,modclass
0,ISIC_0355023,IP_9623174,male,45.0,lower extremity,unknown,benign,0,0.000303,0
1,ISIC_7508527,IP_5945442,female,35.0,torso,nevus,benign,0,0.001300,0
2,ISIC_2844085,IP_2987599,male,55.0,lower extremity,nevus,benign,0,0.000384,0
3,ISIC_2541969,IP_8039381,male,65.0,upper extremity,unknown,benign,0,0.000424,0
4,ISIC_2324728,IP_0347699,male,20.0,torso,unknown,benign,0,0.000289,0


In [20]:
dfout.to_csv(path/'predictions-resnet50-full.csv', index=False)